In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/chess/games.csv


In [2]:
!pip install chess
!apt-get install -y stockfish

STOCKFISH_PATH = "/usr/games/stockfish"

import chess.engine
engine = chess.engine.SimpleEngine.popen_uci(STOCKFISH_PATH)
print(engine.id)
engine.quit()



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.1/6.1 MB 64.3 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
  Created wheel for chess: filename=chess-1.11.2-py3-none-any.whl size=147776 sha256=f623fbce2915a40450ed36d0d69f3fa58b6ceee93e8245404cc1c4f8eb67f3f9
  Stored in directory: /root/.cache/pip/wheels/fb/5d/5c/59a62d8a695285e59ec9c1f66add6f8a9ac4152499a2be0113
Successfully built chess
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
Suggested packages:
  polyglot xboard | scid
The following NEW packages will be installed:
  stockfish
0 upgraded, 1 newly installed, 0 to remove and 122 not upgraded.
Need to get 24.8 MB of archives.
After this operation, 47.4 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 stockfish amd64 14.1-1 [24.8 MB]
Fetched 24.8 MB in 2s (11.8 MB/s)    
Selecting previously unselected package stockfish.
(Reading database ... 128691 files and 

In [3]:
!pip install faiss-cpu sentence-transformers



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 60.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 31.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 2.0 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 89.4 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.8.93
    Uninstalling nvidia-nvjitlink-cu12-12.8.93:
      Successfully uninstalled nvidia-nvjitlink-cu12-12.8.93
  Attempting uninstall: nvidia-curand-cu12
    Found

In [4]:
# Chess Knowledge Base with FAISS for Move Sequences
# This script creates a vector database from chess games stored as move sequences in a CSV

import os
import chess
import faiss
import numpy as np
import pandas as pd
import pickle
from tqdm import tqdm
from sentence_transformers import SentenceTransformer

# Set up constants
DATASET_PATH = '/kaggle/input/chess/games.csv'  # Your CSV dataset path
OUTPUT_DIR = '/kaggle/working/chess_knowledge_base'
MAX_GAMES = 500000  # Maximum number of games to process
ELO_THRESHOLD = 2000  # Minimum ELO rating to include (if available in your CSV)
MOVES_COLUMN = 'moves'  # Column name that contains move sequences
BATCH_SIZE = 32  # Batch size for processing embeddings

# Create output directory
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Function to extract positions from move sequences
def extract_positions_from_moves(csv_file, moves_column=MOVES_COLUMN, max_games=MAX_GAMES):
    positions = []
    game_count = 0
    
    print(f"Loading CSV file from {csv_file}...")
    # Load CSV - adjust parameters as needed for your specific CSV format
    df = pd.read_csv(csv_file, nrows=max_games)
    
    # Check if ELO columns exist for filtering
    has_elo = 'white_elo' in df.columns and 'black_elo' in df.columns
    
    print(f"Processing {len(df)} games from CSV...")
    
    for idx, row in tqdm(df.iterrows(), total=min(len(df), max_games)):
        if game_count >= max_games:
            break
        
        # Apply ELO threshold if data is available
        if has_elo:
            white_elo = int(row['white_elo']) if pd.notna(row['white_elo']) else 0
            black_elo = int(row['black_elo']) if pd.notna(row['black_elo']) else 0
            if min(white_elo, black_elo) < ELO_THRESHOLD and (white_elo > 0 and black_elo > 0):
                continue
        
        # Get move sequence
        if moves_column not in row or pd.isna(row[moves_column]):
            continue
            
        move_sequence = row[moves_column].strip()
        if not move_sequence:
            continue
            
        # Get additional metadata if available
        opening = row.get('opening', 'Unknown') if 'opening' in row else 'Unknown'
        result = row.get('result', '*') if 'result' in row else '*'
        white_player = row.get('white', f'Player_{idx}_w') if 'white' in row else f'Player_{idx}_w'
        black_player = row.get('black', f'Player_{idx}_b') if 'black' in row else f'Player_{idx}_b'
        
        try:
            # Parse the move sequence
            moves_list = move_sequence.split()
            
            # Set up a new board
            board = chess.Board()
            
            # Keep track of positions
            for move_index, move_san in enumerate(moves_list):
                try:
                    # Parse the move in algebraic notation
                    move = board.parse_san(move_san)
                except ValueError:
                    # If SAN parsing fails, try UCI notation
                    try:
                        move = chess.Move.from_uci(move_san)
                        if move not in board.legal_moves:
                            print(f"Warning: illegal move {move_san} in game {idx}, skipping move")
                            continue
                    except ValueError:
                        print(f"Warning: could not parse move {move_san} in game {idx}, skipping move")
                        continue
                
                # Apply the move
                board.push(move)
                
                # Skip the first few moves to focus on meaningful positions
                if move_index < 4:  # Skip first 4 moves
                    continue
                
                # Skip if we've gone too deep into the game
                if move_index >= 40:  # Don't go beyond move 40
                    break
                
                # Get FEN and last move
                fen = board.fen()
                last_move = move.uci()
                
                positions.append({
                    'fen': fen,
                    'last_move': last_move,
                    'opening': opening,
                    'result': result,
                    'white_player': white_player,
                    'black_player': black_player,
                    'white_elo': white_elo if has_elo else None,
                    'black_elo': black_elo if has_elo else None,
                    'move_number': (move_index // 2) + 1,
                    'turn': 'white' if board.turn else 'black',
                    'game_id': idx
                })
            
            game_count += 1
            if game_count % 1000 == 0:
                print(f"Processed {game_count} games, extracted {len(positions)} positions")
                
        except Exception as e:
            print(f"Error processing game {idx}: {e}")
            continue
            
    print(f"Extracted {len(positions)} positions from {game_count} games")
    return positions

# Function to encode chess positions using a pre-trained embedding model
def encode_positions(positions):
    # Load a pre-trained sentence transformer model
    model = SentenceTransformer('all-MiniLM-L6-v2')
    
    print("Encoding positions...")
    texts = []
    
    # Convert each position to a text description for the encoder
    for pos in tqdm(positions):
        fen = pos['fen']
        # Create a text representation of the board
        board = chess.Board(fen)
        board_str = str(board)
        
        # Add additional context like last move and opening
        text_repr = f"Chess position: {board_str}\nFEN: {fen}\nLast move: {pos.get('last_move', 'none')}"
        if pos.get('opening') and pos['opening'] != 'Unknown':
            text_repr += f"\nOpening: {pos['opening']}"
        texts.append(text_repr)
    
    # Generate embeddings in batches
    vectors = []
    
    for i in tqdm(range(0, len(texts), BATCH_SIZE)):
        batch = texts[i:i+BATCH_SIZE]
        batch_embeddings = model.encode(batch)
        vectors.extend(batch_embeddings)
    
    vectors = np.array(vectors, dtype=np.float32)
    print(f"Created {len(vectors)} vectors with shape {vectors.shape}")
    return vectors

# Function to create and save the FAISS index
def create_faiss_index(vectors, positions):
    print("Creating FAISS index...")
    
    # Create a flat index
    dimension = vectors.shape[1]
    index = faiss.IndexFlatL2(dimension)
    
    # Add vectors to the index
    index.add(vectors)
    print(f"Added {index.ntotal} vectors to the index")
    
    # Save the index
    faiss.write_index(index, os.path.join(OUTPUT_DIR, 'chess_positions.index'))
    print(f"Saved FAISS index to {os.path.join(OUTPUT_DIR, 'chess_positions.index')}")
    
    # Save the metadata
    with open(os.path.join(OUTPUT_DIR, 'positions_metadata.pkl'), 'wb') as f:
        pickle.dump(positions, f)
    print(f"Saved positions metadata to {os.path.join(OUTPUT_DIR, 'positions_metadata.pkl')}")
    
    return index

# Function to add Stockfish evaluations using python-chess engine
def add_stockfish_evaluation(positions, sample_size=1000):
    print("Adding Stockfish evaluations for a sample of positions...")
    
    # Use a subset of positions for efficiency
    if len(positions) > sample_size:
        import random
        positions_sample = random.sample(positions, sample_size)
    else:
        positions_sample = positions
    
    try:
        # Initialize Stockfish engine
        # Note: This assumes Stockfish is installed and in the PATH
        # For Kaggle, you may need to install Stockfish or use a different approach
        try:
            import chess.engine
            engine = chess.engine.SimpleEngine.popen_uci(STOCKFISH_PATH)
            engine_available = True
        except (ImportError, FileNotFoundError):
            print("Stockfish engine not available. Skipping evaluations.")
            engine_available = False
        
        if engine_available:
            for pos in tqdm(positions_sample):
                board = chess.Board(pos['fen'])
                
                # Skip if the game is already over
                if board.is_game_over():
                    pos['stockfish_eval'] = None
                    pos['best_move'] = None
                    continue
                
                # Get engine evaluation
                info = engine.analyse(board, chess.engine.Limit(time=0.1))
                
                # Store evaluation and best move
                score = info["score"].relative.score(mate_score=10000)
                if score is not None:
                    pos['stockfish_eval'] = score / 100.0
                else:
                    pos['stockfish_eval'] = None
                
                pos['best_move'] = info["pv"][0].uci() if len(info.get("pv", [])) > 0 else None
            
            engine.quit()
    except Exception as e:
        print(f"Error setting up Stockfish engine: {e}")
        print("Continuing without Stockfish evaluations")
        
    return positions

# Function to test the index with a sample query
def test_index(index, positions, moves_sequence):
    print(f"Testing index with move sequence: {moves_sequence}")
    
    # Parse the move sequence to get FEN
    board = chess.Board()
    moves_list = moves_sequence.split()
    
    for move_san in moves_list:
        try:
            move = board.parse_san(move_san)
            board.push(move)
        except ValueError:
            try:
                move = chess.Move.from_uci(move_san)
                if move in board.legal_moves:
                    board.push(move)
                else:
                    print(f"Warning: illegal move {move_san}, skipping")
            except ValueError:
                print(f"Warning: could not parse move {move_san}, skipping")
    
    # Get the final FEN
    test_fen = board.fen()
    print(f"Converted to FEN: {test_fen}")
    
    # Create a test object with the query FEN
    test_pos = {'fen': test_fen}
    
    # Encode the query position
    query_vector = encode_positions([test_pos])[0].reshape(1, -1)
    
    # Search the index
    k = 5  # Number of nearest neighbors to retrieve
    distances, indices = index.search(query_vector, k)
    
    # Print results
    print(f"Found {len(indices[0])} similar positions")
    for i, idx in enumerate(indices[0]):
        if idx < len(positions):
            pos = positions[idx]
            print(f"Match {i+1}: Distance = {distances[0][i]:.4f}")
            print(f"  FEN: {pos['fen']}")
            print(f"  Last move: {pos['last_move']}")
            print(f"  Opening: {pos['opening']}")
            if 'stockfish_eval' in pos and pos['stockfish_eval'] is not None:
                print(f"  Stockfish evaluation: {pos['stockfish_eval']:.2f}")
                print(f"  Best move: {pos['best_move']}")
            print()

# Main function to build the knowledge base
def build_knowledge_base():
    print("Starting knowledge base creation process...")
    
    # Extract positions from move sequences in CSV
    positions = extract_positions_from_moves(DATASET_PATH)
    
    # Save raw positions data
    with open(os.path.join(OUTPUT_DIR, 'raw_positions.pkl'), 'wb') as f:
        pickle.dump(positions, f)
    print(f"Saved raw positions to {os.path.join(OUTPUT_DIR, 'raw_positions.pkl')}")
    
    # Optional: Add Stockfish evaluations for a sample
    positions = add_stockfish_evaluation(positions)
    
    # Encode positions
    vectors = encode_positions(positions)
    
    # Create and save FAISS index
    index = create_faiss_index(vectors, positions)
    
    # Test the index with a sample position
    test_moves = "d4 d5 c4 c6 cxd5 e6 dxe6 fxe6 Nf3 Bb4+ Nc3 Ba5 Bf4"  # Example from your message
    test_index(index, positions, test_moves)
    
    print(f"Knowledge base created successfully. Files saved to {OUTPUT_DIR}")
    print("You can download the following files from Kaggle:")
    print(f"1. {os.path.join(OUTPUT_DIR, 'chess_positions.index')} - FAISS index file")
    print(f"2. {os.path.join(OUTPUT_DIR, 'positions_metadata.pkl')} - Position metadata")
    print(f"3. {os.path.join(OUTPUT_DIR, 'raw_positions.pkl')} - Raw position data")

# Run the knowledge base building process
if __name__ == "__main__":
    build_knowledge_base()
    

2025-04-19 12:03:09.190509: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1745064189.377140      31 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1745064189.440201      31 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Starting knowledge base creation process...
Loading CSV file from /kaggle/input/chess/games.csv...
Processing 20058 games from CSV...


  5%|▌         | 1087/20058 [00:02<00:42, 443.51it/s]

Processed 1000 games, extracted 31690 positions


 10%|█         | 2020/20058 [00:04<00:39, 453.08it/s]

...

Created 636970 vectors with shape (636970, 384)
Creating FAISS index...
Added 636970 vectors to the index
Saved FAISS index to /kaggle/working/chess_knowledge_base/chess_positions.index
Saved positions metadata to /kaggle/working/chess_knowledge_base/positions_metadata.pkl
Testing index with move sequence: d4 d5 c4 c6 cxd5 e6 dxe6 fxe6 Nf3 Bb4+ Nc3 Ba5 Bf4
Converted to FEN: rnbqk1nr/pp4pp/2p1p3/b7/3P1B2/2N2N2/PP2PPPP/R2QKB1R b KQkq - 5 7
Encoding positions...


  0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

100%|██████████| 1/1 [00:00<00:00, 50.25it/s]

Created 1 vectors with shape (1, 384)
Found 5 similar positions
Match 1: Distance = 0.0518
  FEN: rn1qk2r/pppb1ppp/3b4/3np3/1P6/PQ3N2/3PPPPP/RNB1KB1R b KQkq - 5 7
  Last move: a4b3
  Opening: Unknown

Match 2: Distance = 0.0524
  FEN: rnbqkb1r/pp3ppp/5n2/3pp3/2P5/1P1P1N1P/P4PP1/RNBQKB1R b KQkq - 0 7
  Last move: b2b3
  Opening: Unknown

Match 3: Distance = 0.0569
  FEN: rn1qkb1r/p1pb1ppp/4pn2/3p4/3P4/1Q6/PP2PPPP/RNB1KBNR b KQkq - 3 7
  Last move: b5b3
  Opening: Unknown

Match 4: Distance = 0.0569
  FEN: rnbqkb1r/1p1p1ppp/p4n2/2p5/2P5/2N1B3/PP3PPP/R2QKBNR b KQkq - 1 7
  Last move: b5c3
  Opening: Unknown

Match 5: Distance = 0.0571
  FEN: rnb1k1nr/pp4pp/1qpbpp2/3p4/2PP3B/1PN2N2/P3PPPP/R2QKB1R b KQkq - 0 7
  Last move: b2b3
  Opening: Unknown

Knowledge base created successfully. Files saved to /kaggle/working/chess_knowledge_base
You can download the following files from Kaggle:
1. /kaggle/working/chess_knowledge_base/chess_positions.index - FAISS index file
2. /kaggle/working/chess_kn

In [6]:
import shutil
from google.colab import files  # works the same in Kaggle

# Zip the folder (replace 'my_folder' with your folder name)
shutil.make_archive('my_folder', 'zip', '/kaggle/working/chess_knowledge_base')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>